In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/riiid-test-answer-prediction'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import riiideducation

from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, learning_curve
import lightgbm as lgb
from lightgbm import LGBMClassifier
import eli5



%matplotlib inline
# for heatmap and other plots
colorMap1 = sns.color_palette("RdBu_r")
# for countplot and others plots
colorMap2 = 'Blues_r'

from pandas.api.types import is_datetime64_any_dtype as is_datetime
from pandas.api.types import is_categorical_dtype

In [ ]:
train_path = "/kaggle/input/riiid-test-answer-prediction/train.csv"
questions_path = "../input/riiid-test-answer-prediction/questions.csv"
lectures_path = "../input/riiid-test-answer-prediction/lectures.csv"
test = "../input/riiid-test-answer-prediction/example_test.csv"

# **DATA EXPLORATION & EDA**

We used the 5% of the data

In [ ]:
%%time

train = pd.read_csv(train_path, low_memory=False,  nrows=10123033,
                       dtype={'row_id': 'int64', 'timestamp': 'int64', 'user_id': 'int32', 'content_id': 'int16', 'content_type_id': 'int8',
                              'task_container_id': 'int16', 'user_answer': 'int8', 'answered_correctly': 'int8', 'prior_question_elapsed_time': 'float32', 
                             'prior_question_had_explanation': 'boolean',
                             }
                      )
train

In [ ]:
print(f"Train shape: {train.shape}")

In [ ]:
train.memory_usage()

In [ ]:
train.head(10)

In [ ]:
train.tail()

In [ ]:
train.columns

In [ ]:
train.info()

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(train['answered_correctly'], bins=100)
plt.xlabel('answered_correctly')
plt.ylabel('Fréquence')

In [ ]:
plt.figure(figsize=(15,7))
plt.hist(train['user_answer'], bins=100)
plt.xlabel('user_answer')
plt.ylabel('Fréquence')

In [ ]:
merge_plot = (train['answered_correctly'].groupby(train['user_answer']))
sns.barplot(x='answered_correctly', y='user_answer', data=train)
plt.title('answered_correctly for user_answer')

# Missing Values:

In [ ]:
train.isnull()

In [ ]:
print(train.isnull().sum())

In [ ]:
train.isnull().sum()*100/train.shape[0]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
percent = (train.isnull().sum()*100/train.shape[0]).sort_values(ascending=False)
percent.plot(kind="bar", figsize = (20,10), fontsize = 20)
plt.xlabel("Columns", fontsize = 20)
plt.ylabel("Value Percent(%)", fontsize = 20)
plt.title("Total Missing Value ", fontsize = 20)

Tester les methodes de traitement des valeurs nulles:

In [ ]:
#La suppression:

In [ ]:
train.dropna(axis=0)

In [ ]:
#Remplacement par une valeur factice :

In [ ]:
train['prior_question_elapsed_time'].fillna(0,inplace=True)
train.head()

In [ ]:
train=train.dropna()

In [ ]:
print(train.shape)

In [ ]:
print(train.isnull().sum())

In [ ]:
train.memory_usage()

In [ ]:
#Les variables catégoriques--> Aucune
train.dtypes==object

In [ ]:
train.drop(['row_id', 'timestamp'], axis=1, inplace=True)

In [ ]:
train.describe().style.background_gradient(cmap='Blues')

In [ ]:
train.describe().transpose()

In [ ]:
train['answered_correctly'].describe()

In [ ]:
sns.distplot(train['answered_correctly']);

In [ ]:
train['answered_correctly'].hist()

# Features engineering


Dans l'approche Features Engineering , on essaye de garder les caractéristiques les plus pertinents et les plus utils dans notre étude , pour cela on va dans un premier moment , on va faire une étude de corrélation pour les données séparés , puis on va ajouter d'autres colomne et fonctionalites qu'on trouve assez pertinentes puis on va appliquer des modèles afin de savoir l'importance des Caractéristiques .

In [ ]:
corr_with_target = train.corr()['answered_correctly'].sort_values(ascending=False)
plt.figure(figsize=(14,7))
corr_with_target.drop('answered_correctly').plot.bar()
plt.show()

In [ ]:
feature = ['answered_correctly','user_id', 'content_id', 'task_container_id',
       'user_answer', 'prior_question_elapsed_time','prior_question_had_explanation']
df = train[feature]
df.head()

In [ ]:
corr_with_target = df.corr()["answered_correctly"].sort_values(ascending=False)
plt.figure(figsize=(14,7))
corr_with_target.drop("answered_correctly").plot.bar()
plt.show()

In [ ]:
plt.figure(figsize=(25,20))
sns.set(font_scale=1)
sns.heatmap(df.corr(),annot=True,cmap='coolwarm')

In [ ]:
correlation_matrix  = train.corr()
correlation_matrix["answered_correctly"].sort_values(ascending=False)

# Features

I'll give just some part from our data bacause of the RAM limit on Kaggle kernel

In [ ]:
n = int(train.shape[0] * 1)
train_n = train.sample(n=n, random_state=42)
n

In [ ]:
user_characteristics = train.groupby('user_id').agg({'answered_correctly':
                                                  ['mean', 'median', 'std', 'skew', 'count']})
user_characteristics.columns = [
    'mean_user_acc',
    'median_user_acc',
    'std_user_acc',
    'skew_user_acc',
    'number_of_answered_q'
]

In [ ]:
user_characteristics.head(5)

In [ ]:
task_container_characteristics = train.groupby('task_container_id').agg({'answered_correctly':
                                                                      ['mean', 'median', 'std', 'skew', 'count']})
task_container_characteristics.columns = [
    'mean_task_acc',
    'median_task_acc',
    'std_task_acc',
    'skew_task_acc',
    'number_of_asked_task_containers'
]

In [ ]:
task_container_characteristics.head(5)

In [ ]:
content_characteristics = train.groupby('content_id').agg({'answered_correctly':
                                                        ['mean', 'median', 'std', 'skew', 'count']})
content_characteristics.columns = [
    'mean_acc',
    'median_acc',
    'std_acc',
    'skew_acc',
    'number_of_asked_q'
]

In [ ]:
content_characteristics.head(5)

In [ ]:
train = train_n.copy()
del train_n

In [ ]:
train = train.merge(user_characteristics, how='left', on='user_id')
train = train.merge(task_container_characteristics, how='left', on='task_container_id')
train = train.merge(content_characteristics, how='left', on='content_id')

# The target: answered_correctly

Answered_correctly is our target, and we have to predict to probability for an answer to be correct. 

In [ ]:
features = [
    'prior_question_elapsed_time', 
    'prior_question_had_explanation',
    'mean_user_acc',
    'median_user_acc',
    'std_user_acc',
    'skew_user_acc',
    'number_of_answered_q',
    'mean_task_acc',
    'median_task_acc',
    'std_task_acc',
    'skew_task_acc',
    'number_of_asked_task_containers',
    'mean_acc',
    'median_acc',
    'std_acc',
    'skew_acc',
    'number_of_asked_q'
]

target = 'answered_correctly'


Drop features that we are not going to use in our model

In [ ]:
col_to_drop = set(train.columns.values.tolist()).difference(features + [target])
for col in col_to_drop:
    del train[col]

In [ ]:
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].fillna(value=False).astype(bool)
train = train.fillna(value=0.5)

In [ ]:
train = train.replace([np.inf, -np.inf], np.nan)
train = train.fillna(0.5)

In [ ]:
train.head(5)

# Model

In [ ]:
train_df, test_df, y_train, y_test = train_test_split(train[features], train[target], random_state=777, test_size=0.3)

1. **LGBMClassifier**

In [ ]:
# clf = LGBMClassifier(random_state=777)

#params = {
#   'n_estimators': [50, 150, 300],'max_depth': [3, 5, 10],'num_leaves': [5, 15, 30],'min_data_in_leaf': [5, 50, 100],
#    'feature_fraction': [0.1, 0.5, 1.],'lambda': [0., 0.5, 1.],
#}
#cv = RandomizedSearchCV(clf, param_distributions=params, cv=5, n_iter=50, verbose=2)
#cv.fit(train_df, y_train)

#print(cv.best_params_)
#print(cv.best_score_)

In [ ]:
params = {
    'num_leaves': 30, 
    'n_estimators': 300, 
    'min_data_in_leaf': 100, 
    'max_depth': 5, 
    'lambda': 0.0, 
    'feature_fraction': 1.0
}

In [ ]:
model = LGBMClassifier(**params)
model.fit(train_df, y_train)

In [ ]:
print('LGB ROC-AUC score: ', roc_auc_score(y_test.values, model.predict_proba(test_df)[:, 1]))

In [ ]:
eli5.show_weights(model, top=20)

In [ ]:
lgb.plot_importance(model)

2. **LogisticRegression**

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
logreg = LogisticRegression()

In [ ]:
logreg.fit(train_df, y_train)

Predicting the test set results and calculating the accuracy

In [ ]:
y_pred = logreg.predict(test_df)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(test_df, y_test)))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_pred)
print(confusion_matrix)

**The result is telling us that we have 208239+1844304 correct predictions and 804476+108660  incorrect predictions.**

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
#Grid Search
from sklearn.model_selection import GridSearchCV
clf = LogisticRegression()
grid_values = {'penalty': ['l2'],'C':[0.001,.009,0.01,.09,1,5,10,25]}
grid_clf_acc = GridSearchCV(clf, param_grid = grid_values,scoring = 'recall')
grid_clf_acc.fit(train_df, y_train)

#Predict values based on new parameters
y_pred_acc = grid_clf_acc.predict(test_df)

# New Model Evaluation metrics 
print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred_acc)))
print('Precision Score : ' + str(precision_score(y_test,y_pred_acc)))
print('Recall Score : ' + str(recall_score(y_test,y_pred_acc)))
print('F1 Score : ' + str(f1_score(y_test,y_pred_acc)))

#Logistic Regression (Grid Search) Confusion matrix
confusion_matrix(y_test,y_pred_acc)

# **The support is the number of occurrences of each class in y_test.**

ROC Curve

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, logreg.predict(test_df))
fpr, tpr, thresholds = roc_curve(y_test, logreg.predict_proba(test_df)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('Log_ROC')
plt.show()

The receiver operating characteristic (ROC) curve is another common tool used with binary classifiers. The dotted line represents the ROC curve of a purely random classifier; a good classifier stays as far away from that line as possible (toward the top-left corner).

3. **RandomForestClassifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(train_df, y_train)
clf_pred = clf.predict(test_df)
print('Accuracy of Random Forest Classifier on test set: {:.2f}'.format(clf.score(test_df, y_test)))

In [ ]:
n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(forest, hyperF, cv = 3, verbose = 1, n_jobs = -1)
bestF = gridF.fit(train_df, y_train)

In [ ]:
forestOpt = RandomForestClassifier(random_state = 1, max_depth = 15,     
                                   n_estimators = 500, min_samples_split = 2, min_samples_leaf = 1)
                                   
modelOpt = forestOpt.fit(train_df, y_train)
y_pred = modelOpt.predict(test_df)

In [ ]:
clf_roc_auc = roc_auc_score(y_test, clf.predict(test_df))
fpr, tpr, thresholds = roc_curve(y_test, clf.predict_proba(test_df)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Random Forest Classifier(area = %0.2f)' % clf_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
plt.savefig('clf_ROC')
plt.show()

* gridsearch pour les parametres. essayer la summission

In [ ]:
from sklearn.ensemble import VotingClassifier

clf1 = LogisticRegression(max_iter=1000, random_state=123)
clf2 = RandomForestClassifier(n_estimators=100, random_state=123)
clf3 = GaussianNB()
X = np.array([[-1.0, -1.0], [-1.2, -1.4], [-3.4, -2.2], [1.1, 1.2]])
y = np.array([1, 1, 2, 2])

eclf = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3)],
                        voting='soft',
                        weights=[1, 1, 5])

# predict class probabilities for all classifiers
probas = [c.fit(X, y).predict_proba(X) for c in (clf1, clf2, clf3, eclf)]

# get class probabilities for the first sample in the dataset
class1_1 = [pr[0, 0] for pr in probas]
class2_1 = [pr[0, 1] for pr in probas]


# plotting

N = 4  # number of groups
ind = np.arange(N)  # group positions
width = 0.35  # bar width

fig, ax = plt.subplots()

# bars for classifier 1-3
p1 = ax.bar(ind, np.hstack(([class1_1[:-1], [0]])), width,
            color='green', edgecolor='k')
p2 = ax.bar(ind + width, np.hstack(([class2_1[:-1], [0]])), width,
            color='lightgreen', edgecolor='k')

# bars for VotingClassifier
p3 = ax.bar(ind, [0, 0, 0, class1_1[-1]], width,
            color='blue', edgecolor='k')
p4 = ax.bar(ind + width, [0, 0, 0, class2_1[-1]], width,
            color='steelblue', edgecolor='k')

# plot annotations
plt.axvline(2.8, color='k', linestyle='dashed')
ax.set_xticks(ind + width)
ax.set_xticklabels(['LogisticRegression\nweight 1',
                    'GaussianNB\nweight 1',
                    'RandomForestClassifier\nweight 5',
                    'VotingClassifier\n(average probabilities)'],
                   rotation=40,
                   ha='right')
plt.ylim([0, 1])
plt.title('Class probabilities for sample 1 by different classifiers')
plt.legend([p1[0], p2[0]], ['class 1', 'class 2'], loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
leaderboard  = pd.DataFrame({'Accuracy':[2.80,1.41, 1.78]},index = ['Logistic Regression',
                                                                    'Random Forest', 'LGBMClassifier'])
fig_dims = (15, 8)

fig, a = plt.subplots(figsize=fig_dims)
ax = sns.barplot(x="Accuracy", y=leaderboard.index, data=leaderboard,ax=a)

# SUBMISSION 

In [ ]:
env = riiideducation.make_env()


In [ ]:
iter_test = env.iter_test()


In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    # merge
    test_df = test_df.merge(user_characteristics, on = "user_id", how = "left")
    test_df = test_df.merge(task_container_characteristics, on = "task_container_id", how = "left")
    test_df = test_df.merge(content_characteristics, on = "content_id", how = "left")
    
    # type transformation
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(value=False).astype(bool)
    test_df.fillna(value = 0.5, inplace = True)
    test_df = test_df.replace([np.inf, -np.inf], np.nan)
    test_df = test_df.fillna(0.5)
    
    # preds
    test_df['answered_correctly'] = model.predict_proba(test_df[features])[:, 1]
    cols_to_submission = ['row_id', 'answered_correctly', 'group_num']
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])